In [20]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pywt
import os
import copy

# Frame Extraction

In [16]:

def extract_frames(video_path, output_folder):
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    video_capture = cv2.VideoCapture(video_path)
    fps = video_capture.get(cv2.CAP_PROP_FPS)

    if not video_capture.isOpened():
        print(f"Error: Could not open video {video_path}")
        return
    
    frame_count = 0
    while True:
        ret, frame = video_capture.read()
        if not ret:
            break

        frame_filename = os.path.join(output_folder, f"frame_{frame_count:04d}.png")
        cv2.imwrite(frame_filename, frame)
        
        print(f"Extracted frame {frame_count}")
        frame_count += 1
    
    video_capture.release()
    print("Frame extraction completed.")
    return fps

frame_rate = extract_frames('testVid.mp4', 'extracted_frames')


Extracted frame 0
Extracted frame 1
Extracted frame 2
Extracted frame 3
Extracted frame 4
Extracted frame 5
Extracted frame 6
Extracted frame 7
Extracted frame 8
Extracted frame 9
Extracted frame 10
Extracted frame 11
Extracted frame 12
Extracted frame 13
Extracted frame 14
Extracted frame 15
Extracted frame 16
Extracted frame 17
Extracted frame 18
Extracted frame 19
Extracted frame 20
Extracted frame 21
Extracted frame 22
Extracted frame 23
Extracted frame 24
Extracted frame 25
Extracted frame 26
Extracted frame 27
Extracted frame 28
Extracted frame 29
Extracted frame 30
Extracted frame 31
Extracted frame 32
Extracted frame 33
Extracted frame 34
Extracted frame 35
Extracted frame 36
Extracted frame 37
Extracted frame 38
Extracted frame 39
Extracted frame 40
Extracted frame 41
Extracted frame 42
Extracted frame 43
Extracted frame 44
Extracted frame 45
Extracted frame 46
Extracted frame 47
Extracted frame 48
Extracted frame 49
Extracted frame 50
Extracted frame 51
Extracted frame 52
Ext

# Helper Functions


## Watermark

In [ ]:
def create_watermark(shape):
    return np.random.randint(0, 256, shape, dtype=np.uint8)

## DWT

In [23]:
def dwt_transform(image, wavelet = 'haar'):
    
    b, g, r = cv2.split(image)

    coeffsb = pywt.dwt2(b, wavelet)
    coeffsg = pywt.dwt2(g, wavelet)
    coeffsr = pywt.dwt2(r, wavelet)
    
    return cv2.merge((coeffsb[0], coeffsg[0], coeffsr[0]))


def dwt_transform_inverse(coeffs , wavelet = 'haar'):

    coeffsB = (coeffs[0][:, :, 0], 
               (coeffs[1][0][:, :, 0],
               coeffs[1][1][:, :, 0],
               coeffs[1][1][:, :, 0]))
    coeffsG = (coeffs[0][:, : , 1],
               (coeffs[1][0][:, :, 1],
                coeffs[1][1][:, :, 1],
                coeffs[1][1][:, :, 1]))
    coeffsR = (coeffs[0][:, : , 2],
               (coeffs[1][0][:, :, 2],
                coeffs[1][1][:, :, 2],
                coeffs[1][1][:, :, 2]))
    
    Ib = pywt.idwt2(coeffsB, wavelet=wavelet)
    Ig = pywt.idwt2(coeffsG, wavelet=wavelet)
    Ir = pywt.idwt2(coeffsR, wavelet=wavelet)

    return cv2.merge(Ib, Ig, Ir)

## Chaotic Map

In [31]:
def CFold_1(Z):
        return Z.real % 1 + (Z.imag % 1) * 1j

def compute_Z1(a, c1, Z1, Z2):
    return CFold_1(a * ((Z1 / Z2)**2)) + c1 

def compute_Z2(b, c2, Z1, Z2):    
    return CFold_1(b * ((Z2 / Z1)**2)) + c2 

def compute_x_y(Z1, Z2, rcB4, ccB4):
    return (np.floor(Z1.real * (10**14)) % rcB4,
            np.floor(Z2.imag * (10**14)) % ccB4)

def compute_channel(Z2, channels = 3):
    return np.floor(Z2.real * (10**14)) % channels

def compute_B(Z2):
    return np.floor(Z2[-1].imag * (10**14)) % 2

## Embedding Conditions

In [33]:
def condition_for_zero(Wij, U, T):
    return (Wij == 0  and 
            np.abs(U[0, 0]) > np.abs(U[1, 0]) and 
            np.abs(U[1, 0]) - np.abs(U[0, 0]) < T)

def condition_for_one(Wij, U, T):
    return (Wij == 1 and  
            np.abs(U[0, 0]) < np.abs(U[1, 0]) and 
            np.abs(U[1, 0]) - np.abs(U[0, 0]) > T)

## Inverse SVD

In [2]:
def inverse_svd(U, S, Vt):
    S = np.diag(S)
    S_full = np.zeros((U.shape[0], Vt.shape[1]))
    S_full[:S.shape[0], :S.shape[1]] = S
    return U @ S_full @ Vt

# Embedding and Extraction Function

In [ ]:
def embedding_extraction_functions(I = None, W = None, param = None, extract = True):

    if I is None:
        raise Exception("Frame not provided.")
    elif W is None:
        raise Exception("Watermark not provided.")
    elif param is None:
        raise Exception("Parameters not provided.")
    
    if extract:
        W_extract = np.zeros_like(W)
    
    LL, (LH, HL, HH) = dwt_transform(I)    
    rc, cc, channels = LL.shape
    rcB4, ccB4 = rc // 4, cc // 4

    mask = np.zeros((rcB4, ccB4, channels))     

    for i in range(W.shape[0]):
        for j in range(W.shape[1]):
            
            while True:
                
                param['Z1_'].append(
                    compute_Z1(param['a'], param['c1'], 
                               param['Z1_'][-1], param['Z2_'][-1]))
                
                param['Z2_'].append(
                    compute_Z2(param['b'], param['c2'], 
                               param['Z1_'][-1], param['Z2_'][-1]))
    
                x, y = compute_x_y(param['Z1_'][-1], 
                                   param['Z2_'][-1], 
                                   rcB4, ccB4)
    
                channel = compute_channel(param['Z2_'][-1])
    
                B = compute_B(param['Z2_'][-1])
                
                if mask[x, y, channel] == 0:
                    break

        block = LL[x * 4 : x * 4 + 4, 
                    y * 4 : y * 4 + 4,
                    channel]
        
        U, S, Vt = np.linalg.svd(block, full_matrices = True)

        # EMBEDDING
        if not extract:
            
            meanV = (np.abs(U[0, 0]) + np.abs(U[1, 0])) / 2

            if condition_for_zero(W[i, j], U, param['T']):
                U[0, 0] = np.sign(U[0, 0]) * (meanV + param['T'] / 2)
                U[1, 0] = np.sign(U[1, 0]) * (meanV - param['T'] / 2)

            # XOR
            # if Wij != B then Wij = 1 and block below executes
            W[i, j] = W[i, j] ^ B 
            
            if condition_for_one(W[i, j, U, param['T']]): 
                U[0, 0] = np.sign(U[0, 0]) * (meanV - param['T'] / 2)
                U[1, 0] = np.sign(U[1, 0]) * (meanV + param['T'] / 2)

            LL[x * 4 : x * 4 + 4, 
               y * 4 : y * 4 + 4,
               channel] = inverse_svd(U, S, Vt)

        # EXTRACTION
        if extract:

            if np.abs(U[0, 0]) > np.abs(U[1, 0]):
                W_extract[i,j] = 0
            else:
                W_extract[i,j] = 1

            W_extract[i,j] = W_extract[i,j] ^ B

        mask[x, y, channel] = 1
    
    if not extract:
        return dwt_transform_inverse((LL, (LH, HL, HH)))
    elif extract:
        return W_extract

        

# Embed Frames

In [19]:
def embed(frames_folder, param, watermark_shape):

    frame_paths = [frm for frm in os.listdir(frames_folder) if frm.endswith(".png")]
    frame_paths.sort()
    
    param_copy = copy.deepcopy(param)
    
    watermark = create_watermark(watermark_shape)

    embeded_frames = []

    for frame_path in frame_paths:
        path = os.path.join(frames_folder, frame_path)

        embeded_frames.append(
            embedding_extraction_function(
                I = cv2.imread(path), 
                W = watermark,
                param = param_copy,
                extract = False))
    
    return embeded_frames, watermark, param_copy 

In [ ]:
param_initial = {
        'a' : 6, 
        'b' : 4,
        'Z1_' : [0.4 + 0.1j],
        'Z2_' : [0.6 + 0.3j],
        'c1' : 0.2, 
        'c2' : 0.7, 
        'T' : 1
    }


embeded_frames, watermark, param_final = embed(frames_folder='extracted_frames', 
                                               param=param_initial, 
                                               watermark_shape=(10, 10))

# Save as Video

In [18]:
def save_frames(frames, output_folder):
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for frame_count, frame in enumerate(frames):
        frame_filename = os.path.join(output_folder, f"frame_{frame_count:04d}.png")
        cv2.imwrite(frame_filename, frame)

        print(f"Saved Frame {frame_count}")

def create_video_from_images(image_folder, output_video_path, frame_rate):
    
    images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
    images.sort()  
    
    if not images:
        print("No images found in the folder.")
        return
    
    first_image_path = os.path.join(image_folder, images[0])
    frame = cv2.imread(first_image_path)
    height, width, channels = frame.shape
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  
    video_writer = cv2.VideoWriter(output_video_path, fourcc, frame_rate, (width, height))
    
    for image in images:
        image_path = os.path.join(image_folder, image)
        frame = cv2.imread(image_path)
        
        if frame is None:
            print(f"Error reading image {image_path}")
            continue
        
        video_writer.write(frame)
        print(f"Writing frame {image_path}")
    
    video_writer.release()
    print(f"Video creation completed. Saved to {output_video_path}")


In [ ]:
save_frames(frames = embeded_frames, 
            output_folder='embeded_frames')

In [ ]:

create_video_from_images(image_folder='extracted_frames', 
                         output_video_path='embeded_video.mp4', 
                         frame_rate = frame_rate)